In [6]:
import sys
import bs4 as bs
import urllib.request
import requests
import pandas as pd
from io import StringIO
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
import urllib.parse
from lxml.html.clean import Cleaner
from datetime import datetime
import posixpath

# Concurrency
from concurrent.futures import ThreadPoolExecutor
from concurrent.futures import as_completed

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [3]:
# https://lxml.de/api/lxml.html.clean.Cleaner-class.html 
def sanitize(bloated_html):
    cleaner = Cleaner(page_structure=True,
                  meta=True,
                  embedded=True,
                  links=True,
                  style=True,
                  processing_instructions=True,
                  inline_style=True,
                  scripts=True,
                  javascript=True,
                  comments=True,
                  frames=True,
                  forms=True,
                  annoying_tags=True,
                  remove_unknown_tags=True,
                  safe_attrs_only=True,
                  safe_attrs=frozenset(['src','color', 'href', 'title', 'class', 'name', 'id']),
                  remove_tags=('span', 'font', 'div')
                  )

    return cleaner.clean_html(bloated_html)

# Break pandas single column into multiple cols based on rows
def get_rows_to_break(df):
    # get the row where the date ends for numpy to reshape
    for i, val in enumerate(df.values[:-1]):
        if i > 1: # ignore the first two rows as they are non-datetime headers
            try:
                result = datetime.strptime(val[0],"%m/%d/%Y")
            except ValueError:
                break
    return i

In [4]:
def get_financial_data(ticker, statement_type, expanded=False, quarterly=False):
    # set header for requests library
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'}
    
    # set headless mode and initialize firefox headless mode
    options = Options()
    options.headless = True
    driver = webdriver.Firefox(options=options)

    # set base url
    base_url = "https://finance.yahoo.com/quote/"

    if statement_type == "CF":
        new_url = posixpath.join(base_url, ticker, "cash-flow")
    elif statement_type == "BS":
        new_url = posixpath.join(base_url, ticker, "balance-sheet")
    elif statement_type == "INC": # getting income statement
        new_url = posixpath.join(base_url, ticker, "financials")
    
    print(new_url)
    # use selenium to extract
    if new_url and expanded==True:
        # get new url from the webpage
        driver.get(new_url)
        
        s = requests.Session()
        
        # click on the "Expand ALL" button on finance yahoo
        # sbtn = driver.find_element_by_css_selector('button.expandPf')
        parent = driver.find_element(By.XPATH, "//span[text()='Expand All']")
        sbtn = parent.find_element(By.XPATH, '..').find_element(By.XPATH, '..')

        sbtn.click()
        
        if quarterly == True: # extract quarterly data
            
            # get button with quarterly on it
            parent = driver.find_element(By.XPATH, "//span[text()='Quarterly']")
            
            # get parent element of span element which is the div and then the button
            qbtn = parent.find_element(By.XPATH, '..').find_element(By.XPATH,'..')
            
            qbtn.click()

        html_from_page = driver.page_source
        soup = bs.BeautifulSoup(html_from_page, 'html.parser').find_all("div", class_="M(0) Whs(n) BdEnd Bdc($seperatorColor) D(itb)")
        
    elif new_url and expanded==False:
        response = requests.get(new_url, headers=headers)
                                
        soup = bs.BeautifulSoup(response.content, 'html5lib').find_all("div", class_="M(0) Whs(n) BdEnd Bdc($seperatorColor) D(itb)")
    
    else:
        raise Exception("No New url given")
    
    s = sanitize(soup[0].prettify()).replace("&amp;","&") # remove unnecessary elements
    df = pd.read_csv(StringIO(s), sep=r"\n\n+")
    # print(df)
    df1 = df.values[:-1].reshape(-1, get_rows_to_break(df))
    # print(df1)
    driver.close()
    driver.quit()
    
    return pd.DataFrame(df1[1:], columns=df1[0,0:]).set_index(['Breakdown'])

In [7]:
def get_allstatements(ticker, statements):
    """
    Create a thread pool and download specified urls
    """
    final_statements = {}
    with ThreadPoolExecutor(max_workers=3) as executor:
        futures = {executor.submit(get_financial_data, 
                                   ticker, 
                                   statement, 
                                   expanded=expd, 
                                   quarterly=quart): statement for (statement, expd, quart) in statements}
        
        for future in as_completed(futures):
            final_statements[futures[future]] = future.result()
    
    return final_statements
            
if __name__ == '__main__':
    FB_statement = get_allstatements("FB", [("INC", True, True),("BS", True, False), ("CF", True, False)])

https://finance.yahoo.com/quote/FB/financials
https://finance.yahoo.com/quote/FB/cash-flow
https://finance.yahoo.com/quote/FB/balance-sheet


/home/andante/.local/lib/python3.9/site-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [8]:
FB_statement['INC']

,ttm,12/30/2021,9/29/2021,6/29/2021,3/30/2021,12/30/2020
Breakdown,,,,,,
Total Revenue,"117,929,000","33,671,000","29,010,000","29,077,000","26,171,000","28,072,000"
Operating Revenue,"117,929,000","35,635,000","28,276,000","28,580,000","25,439,000","27,188,000"
Cost of Revenue,"22,649,000","6,348,000","5,771,000","5,399,000","5,131,000","5,210,000"
Gross Profit,"95,280,000","27,323,000","23,239,000","23,678,000","21,040,000","22,862,000"
Operating Expense,"48,527,000","14,738,000","12,816,000","11,311,000","9,662,000","10,087,000"
Selling General and Administrative,"23,872,000","7,692,000","6,500,000","5,215,000","4,465,000","4,880,000"
General & Administrative Expense,"9,829,000","3,305,000","2,946,000","1,956,000","1,622,000","1,599,000"
Other G and A,"9,829,000","3,305,000","2,946,000","1,956,000","1,622,000","1,599,000"
Selling & Marketing Expense,"14,043,000","4,387,000","3,554,000","3,259,000","2,843,000","3,281,000"


In [9]:
FB_statement['BS']

,12/30/2021,12/30/2020,12/30/2019,12/30/2018
Breakdown,,,,
Total Assets,"165,987,000","159,316,000","133,376,000","97,334,000"
Current Assets,"66,666,000","75,670,000","66,225,000","50,480,000"
"Cash, Cash Equivalents & Short Term Investments","47,998,000","61,954,000","54,855,000","41,114,000"
Cash And Cash Equivalents,"16,601,000","17,576,000","19,079,000","10,019,000"
Cash,"7,308,000","6,488,000","4,735,000","2,713,000"
Cash Equivalents,"9,293,000","11,088,000","14,344,000","7,306,000"
Other Short Term Investments,"31,397,000","44,378,000","35,776,000","31,095,000"
Receivables,"14,039,000","11,335,000","9,518,000","7,587,000"
Accounts receivable,"14,039,000","11,335,000","9,518,000","7,587,000"


In [10]:
FB_statement['CF']

,ttm,12/30/2021,12/30/2020,12/30/2019,12/30/2018
Breakdown,,,,,
Operating Cash Flow,"57,683,000","57,683,000","38,747,000","36,314,000","29,274,000"
Cash Flow from Continuing Operating Activities,"57,683,000","57,683,000","38,747,000","36,314,000","29,274,000"
Net Income from Continuing Operations,"39,370,000","39,370,000","29,146,000","18,485,000","22,112,000"
Depreciation Amortization Depletion,"7,967,000","7,967,000","6,862,000","5,741,000","4,315,000"
Depreciation & amortization,"7,967,000","7,967,000","6,862,000","5,741,000","4,315,000"
Deferred Tax,"609,000","609,000","-1,192,000","-37,000","286,000"
Deferred Income Tax,"609,000","609,000","-1,192,000","-37,000","286,000"
Stock based compensation,"9,164,000","9,164,000","6,536,000","4,836,000","4,152,000"
Other non-cash items,"-127,000","-127,000","118,000","39,000","-64,000"


In [11]:
FB_statement['BS'].loc[['Total Debt']]

,12/30/2021,12/30/2020,12/30/2019,12/30/2018
Breakdown,,,,
Total Debt,"13,873,000","10,654,000","10,601,000","500,000"


In [12]:
FB_statement['BS'].at['Total Debt', '12/30/2020']

'10,654,000'